# Part 1. #
## This solution makes use of code for wiki table to csv scraping developed by Andy Roche https://github.com/rocheio/wiki-table-scrape ##

In [1]:
#Code for scraping tables to csv

import csv
import os

from bs4 import BeautifulSoup
import requests


def scrape(url, output_name):
    """Create CSVs from all tables in a Wikipedia article.
    ARGS:
        url (str): The full URL of the Wikipedia article to scrape tables from.
        output_name (str): The base file name (without filepath) to write to.
    """

    # Read tables from Wikipedia article into list of HTML strings
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "lxml")
    table_classes = {"class": ["sortable", "plainrowheaders"]}
    wikitables = soup.findAll("table", table_classes)

    # Create folder for output if it doesn't exist
    os.makedirs(output_name, exist_ok=True)

    for index, table in enumerate(wikitables):
        # Make a unique file name for each CSV
        if index == 0:
            filename = output_name
        else:
            filename = output_name + "_" + str(index)

        filepath = os.path.join(output_name, filename) + ".csv"

        with open(filepath, mode="w", newline="", encoding="utf-8") as output:
            csv_writer = csv.writer(output, quoting=csv.QUOTE_ALL, lineterminator="\n")
            write_html_table_to_csv(table, csv_writer)


def write_html_table_to_csv(table, writer):
    """Write HTML table from Wikipedia to a CSV file.
    ARGS:
        table (bs4.Tag): The bs4 Tag object being analyzed.
        writer (csv.writer): The csv Writer object creating the output.
    """

    # Hold elements that span multiple rows in a list of
    # dictionaries that track 'rows_left' and 'value'
    saved_rowspans = []
    for row in table.findAll("tr"):
        cells = row.findAll(["th", "td"])

        # If the first row, use it to define width of table
        if len(saved_rowspans) == 0:
            saved_rowspans = [None for _ in cells]
        # Insert values from cells that span into this row
        elif len(cells) != len(saved_rowspans):
            for index, rowspan_data in enumerate(saved_rowspans):
                if rowspan_data is not None:
                    # Insert the data from previous row; decrement rows left
                    value = rowspan_data["value"]
                    cells.insert(index, value)

                    if saved_rowspans[index]["rows_left"] == 1:
                        saved_rowspans[index] = None
                    else:
                        saved_rowspans[index]["rows_left"] -= 1

        # If an element with rowspan, save it for future cells
        for index, cell in enumerate(cells):
            if cell.has_attr("rowspan"):
                rowspan_data = {"rows_left": int(cell["rowspan"]), "value": cell}
                saved_rowspans[index] = rowspan_data

        if cells:
            # Clean the data of references and unusual whitespace
            cleaned = clean_data(cells)

            # Fill the row with empty columns if some are missing
            # (Some HTML tables leave final empty cells without a <td> tag)
            columns_missing = len(saved_rowspans) - len(cleaned)
            if columns_missing:
                cleaned += [None] * columns_missing

            writer.writerow(cleaned)


def clean_data(row):
    """Clean table row list from Wikipedia into a string for CSV.
    ARGS:
        row (bs4.ResultSet): The bs4 result set being cleaned for output.
    RETURNS:
        cleaned_cells (list[str]): List of cleaned text items in this row.
    """

    cleaned_cells = []

    for cell in row:
        # Strip references from the cell
        references = cell.findAll("sup", {"class": "reference"})
        if references:
            for ref in references:
                ref.extract()

        # Strip sortkeys from the cell
        sortkeys = cell.findAll("span", {"class": "sortkey"})
        if sortkeys:
            for ref in sortkeys:
                ref.extract()

        # Strip footnotes from text and join into a single string
        text_items = cell.findAll(text=True)
        no_footnotes = [text for text in text_items if text[0] != "["]

        cleaned = (
            "".join(no_footnotes)  # Combine elements into single string
            .replace("\xa0", " ")  # Replace non-breaking spaces
            .replace("\n", " ")  # Replace newlines
            .strip()
        )

        cleaned_cells += [cleaned]

    return cleaned_cells

In [2]:
#scraping relevant table
scrape(
    url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M",
    output_name="postal"
)

In [3]:
import pandas as pd
import numpy as np

#reading scraped table from csv file
df=pd.read_csv('postal/postal.csv')
df.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
df2 = df[df.Borough != 'Not assigned'].reset_index(drop=True) #dropping rows with postcodes unassigned to any borough

for n in range(df2.shape[0]):
    if df2.iloc[n,2]=="Not assigned":
        df2.iloc[n,2]=df2.iloc[n,1]    #Not assigned neighbourhood get same names as boroughs
    df2.iloc[n,2]+=', '   #adding comma and space for grouping purpouse later

df2.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,"Parkwoods,"
1,M4A,North York,"Victoria Village,"
2,M5A,Downtown Toronto,"Harbourfront,"
3,M5A,Downtown Toronto,"Regent Park,"
4,M6A,North York,"Lawrence Heights,"
5,M6A,North York,"Lawrence Manor,"
6,M7A,Queen's Park,"Queen's Park,"
7,M9A,Etobicoke,"Islington Avenue,"
8,M1B,Scarborough,"Rouge,"
9,M1B,Scarborough,"Malvern,"


In [5]:
agg_fcns={'Borough':'first', 'Neighbourhood':sum}
df3=df2.groupby(df2['Postcode']).aggregate(agg_fcns).reset_index()   #combining neighborhoods with same postal codes

for n in range(df3.shape[0]):
    df3.iloc[n,2]=df3.iloc[n,2][0:-2]   #removing comma and space at the end of neighborhood names

df3.head(10)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
df3.shape

(103, 3)

# Part 2: Assigning coordinates to postal codes #

In [7]:
latln=pd.read_csv('https://cocl.us/Geospatial_data') #importing coordinates from csv file

In [8]:
latln.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
df4=pd.concat([df3,latln.iloc[:,1:3]], axis=1)
df4.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Part 3 Analyzing data

In [10]:
# importing stuff

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [11]:
# create map of Torronto using latitude and longitude values
latitude=43.653908
longitude=-79.384293
map_torronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, post, hoods in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df['Postcode'], df4['Neighbourhood']):
    label = '{}, {}: {}'.format(post, borough, hoods)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_torronto)  
    
map_torronto

# Please input your Foursquare credentials below!!! #

In [12]:
CLIENT_ID = 'PKTGNOXE1SWZQ5RBENXAZQBBJNA2QCU5J3LYDV2EPFPVE2K1' # your Foursquare ID
CLIENT_SECRET = 'CEHIACT5UXPYNA4HPHLBGGCNOZKR5XUKQ1UTLVZJQ2OAVCLC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PKTGNOXE1SWZQ5RBENXAZQBBJNA2QCU5J3LYDV2EPFPVE2K1
CLIENT_SECRET:CEHIACT5UXPYNA4HPHLBGGCNOZKR5XUKQ1UTLVZJQ2OAVCLC


### Getting venues for each neighbourhood/postcode ###

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
LIMIT=100
RADIUS=500
torronto_venues = getNearbyVenues(names=df4['Neighbourhood'],
                                   latitudes=df4['Latitude'],
                                   longitudes=df4['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [15]:
print(torronto_venues.shape)
torronto_venues.head()

### Determining top 10 most frequent venues for each neighbourhood ###

In [16]:
# one hot encoding
torronto_onehot = pd.get_dummies(torronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
torronto_onehot['Neighborhood'] = torronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [torronto_onehot.columns[-1]] + list(torronto_onehot.columns[:-1])
torronto_onehot = torronto_onehot[fixed_columns]

torronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
torronto_grouped = torronto_onehot.groupby('Neighborhood').mean().reset_index()
torronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.01,0.0,0.000000,0.0,0.0,0.01,0.0,0.0
1,Agincourt,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.090909,0.0,0.0,0.00,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0


In [18]:
num_top_venues = 5

for hood in torronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = torronto_grouped[torronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.05
1                 Café  0.05
2                  Bar  0.04
3  American Restaurant  0.04
4           Steakhouse  0.04


----Agincourt----
                venue  freq
0      Sandwich Place  0.25
1              Lounge  0.25
2      Breakfast Spot  0.25
3  Chinese Restaurant  0.25
4         Yoga Studio  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0          Playground  0.33
1    Asian Restaurant  0.33
2                Park  0.33
3         Yoga Studio  0.00
4  Miscellaneous Shop  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
            venue  freq
0   Grocery Store  0.18
1        Pharmacy  0.09
2      Beer Store  0.09
3    Liquor Store  0.09
4  Sandwich Place  0.09


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place   0.2
1    Skating Rink   0.1
2

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [20]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = torronto_grouped['Neighborhood']

for ind in np.arange(torronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Coffee Shop,Bar,American Restaurant,Steakhouse,Thai Restaurant,Gym,Bakery,Hotel,Burger Joint
1,Agincourt,Chinese Restaurant,Lounge,Sandwich Place,Breakfast Spot,Women's Store,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Asian Restaurant,Playground,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Liquor Store,Pharmacy,Pizza Place,Sandwich Place,Coffee Shop,Video Store
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Skating Rink,Pharmacy,Pub,Dance Studio,Pool,Sandwich Place,Women's Store


### Clustering ###

In [21]:
# set number of clusters
kclusters = 5

torronto_grouped_clustering = torronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 1, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 1, 0, 4,
       4, 4, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 1, 4, 4, 4, 1, 4, 4,
       4, 2, 4, 4, 4, 4, 1, 4, 1, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 1, 4, 4, 4, 1, 4, 4, 4, 4, 4, 1], dtype=int32)

In [22]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted['Cluster Labels']=kmeans.labels_


torronto_merged = df4
neighborhoods_venues_sorted['Postcode']=torronto_merged['Postcode']

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
torronto_merged = torronto_merged.join(neighborhoods_venues_sorted.set_index('Postcode'), on='Postcode')

torronto_merged.tail() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
98,M9N,York,Weston,43.706876,-79.518188,Woodbine Heights,Athletics & Sports,Cosmetics Shop,Beer Store,Skating Rink,Bus Stop,Curling Ice,Park,Pharmacy,Construction & Landscaping,Convenience Store,4.0
99,M9P,Etobicoke,Westmount,43.696319,-79.532242,York Mills West,Park,Convenience Store,Bank,Bar,Women's Store,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,1.0
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,M9W,Etobicoke,Northwest,43.706748,-79.594054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
torronto_merged=torronto_merged.dropna()
torronto_merged.tail()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
95,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.643515,-79.577201,Willowdale West,Pharmacy,Grocery Store,Pizza Place,Coffee Shop,Discount Store,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,4.0
96,M9L,North York,Humber Summit,43.756303,-79.565963,Woburn,Coffee Shop,Korean Restaurant,Convenience Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,4.0
97,M9M,North York,"Emery, Humberlea",43.724766,-79.532242,"Woodbine Gardens, Parkview Hill",Fast Food Restaurant,Pizza Place,Pet Store,Breakfast Spot,Gym / Fitness Center,Pharmacy,Café,Bank,Athletics & Sports,Intersection,4.0
98,M9N,York,Weston,43.706876,-79.518188,Woodbine Heights,Athletics & Sports,Cosmetics Shop,Beer Store,Skating Rink,Bus Stop,Curling Ice,Park,Pharmacy,Construction & Landscaping,Convenience Store,4.0
99,M9P,Etobicoke,Westmount,43.696319,-79.532242,York Mills West,Park,Convenience Store,Bank,Bar,Women's Store,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,1.0


In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
a=1
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torronto_merged['Latitude'], torronto_merged['Longitude'], torronto_merged['Neighborhood'], torronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(int(cluster)), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining Clusters ###

In [25]:
torronto_merged.loc[torronto_merged['Cluster Labels'] == 0, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
42,East Toronto,"Emery, Humberlea",Baseball Field,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store,Dessert Shop,0.0
56,Downtown Toronto,"Humber Bay, King's Mill Park, Kingsway Park So...",Baseball Field,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store,Dessert Shop,0.0


In [26]:
torronto_merged.loc[torronto_merged['Cluster Labels'] == 1, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
13,Scarborough,"CFB Toronto, Downsview East",Park,Other Repair Shop,Airport,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,1.0
16,Scarborough,Caledonia-Fairbanks,Park,Women's Store,Fast Food Restaurant,Market,Pharmacy,Golf Course,Greek Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,1.0
39,East York,"Downsview, North Park, Upwood Park",Basketball Court,Park,Deli / Bodega,Construction & Landscaping,Bakery,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,1.0
41,East Toronto,East Toronto,Convenience Store,Park,Coffee Shop,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,1.0
46,Central Toronto,"Forest Hill North, Forest Hill West",Park,Jewelry Store,Sushi Restaurant,Trail,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Department Store,1.0
47,Central Toronto,Glencairn,Park,Pizza Place,Japanese Restaurant,Pub,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,1.0
59,Downtown Toronto,"Kingsview Village, Martin Grove Gardens, Richv...",Park,Pizza Place,Mobile Phone Shop,Bus Line,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,1.0
63,Central Toronto,Lawrence Park,Park,Bus Line,Swim School,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,1.0
72,North York,Parkwoods,Fast Food Restaurant,Food & Drink Shop,Park,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,1.0
74,York,Rosedale,Park,Trail,Playground,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,1.0


In [27]:
torronto_merged.loc[torronto_merged['Cluster Labels'] == 2, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
2,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Asian Restaurant,Playground,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,2.0
67,Downtown Toronto,"Moore Park, Summerhill East",Tennis Court,Playground,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,2.0
79,North York,Scarborough Village,Playground,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,2.0


In [28]:
torronto_merged.loc[torronto_merged['Cluster Labels'] == 3, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
26,North York,"Cloverdale, Islington, Martin Grove, Princess ...",Bank,Women's Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop,3.0


In [29]:
torronto_merged.loc[torronto_merged['Cluster Labels'] == 4, torronto_merged.columns[[1] + list(range(5, torronto_merged.shape[1]))]]

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Scarborough,"Adelaide, King, Richmond",Café,Coffee Shop,Bar,American Restaurant,Steakhouse,Thai Restaurant,Gym,Bakery,Hotel,Burger Joint,4.0
1,Scarborough,Agincourt,Chinese Restaurant,Lounge,Sandwich Place,Breakfast Spot,Women's Store,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,4.0
3,Scarborough,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Liquor Store,Pharmacy,Pizza Place,Sandwich Place,Coffee Shop,Video Store,4.0
4,Scarborough,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Skating Rink,Pharmacy,Pub,Dance Studio,Pool,Sandwich Place,Women's Store,4.0
5,Scarborough,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Shopping Mall,Bridal Shop,Middle Eastern Restaurant,Supermarket,Bank,Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Diner,4.0
6,Scarborough,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,4.0
7,Scarborough,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Fast Food Restaurant,Pizza Place,Pub,Indian Restaurant,Ice Cream Shop,Sushi Restaurant,Butcher,Japanese Restaurant,4.0
8,Scarborough,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Beer Bar,Italian Restaurant,Seafood Restaurant,Farmers Market,Steakhouse,Café,Bakery,4.0
9,Scarborough,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Comic Shop,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Colombian Restaurant,4.0
10,Scarborough,"Bloordale Gardens, Eringate, Markland Wood, Ol...",Café,Pharmacy,Pizza Place,Beer Store,Liquor Store,Convenience Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,4.0
